In [49]:
import openai

client = openai.OpenAI(api_key='YOUR API TOKEN')

def get_chatgpt_response(prompt):
    """Gets a response from ChatGPT using the OpenAI API"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # The ChatGPT model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,  # Control the creativity of the response
        max_tokens=200  # Limit the length of the response
    )
    return response.choices[0].message.content

In [89]:
import pandas as pd
import numpy as np

generated_modertion_df = pd.read_csv("test_with_fine_tuned_moderations.csv")
generated_modertion_df

,Unnamed: 0,prompt,label,base_model_moderation,fine_tuned_model_moderation
0,0,Assume you are a forum moderator. Please respo...,"SodaPalooza, it's important to remember that t...","SodaPalooza, I understand your perspective, b...","\nSodaPalooza, it's important for you to engag..."
1,1,Assume you are a forum moderator. Please respo...,"TS1307, while your points are well-argued, it'...",BlckJck103: I understand your perspective and...,"\nTS1307, please remember to keep the conversa..."
2,2,Assume you are a forum moderator. Please respo...,"Blindweaponsfumbler, please remember to keep t...","Blindweaponsfumbler, I understand that you ha...","\nBlindweaponsfumbler, please try to focus on ..."
3,3,Assume you are a forum moderator. Please respo...,"Celtabria, it's important to consider the impa...","I understand your perspective, Celtabria. How...","\nCeltabria, please remember to keep the conve..."
4,4,Assume you are a forum moderator. Please respo...,"arden13, it seems like you are providing a tho...","arden13, I understand that this is a complex ...","arden13, please remember to keep the conversat..."
...,...,...,...,...,...
342,342,Assume you are a forum moderator. Please respo...,"Pleberal, please remember to keep the conversa...",I agree with GaslightProphet that the assumpt...,"\nPleberal, please remember to keep the conver..."
343,343,Assume you are a forum moderator. Please respo...,"beend1p, thank you for providing information a...","Trumpologist, it's important to keep the conv...","\nbeend1p, your points have been interesting a..."
344,344,Assume you are a forum moderator. Please respo...,"z3r0shade, it's important to remember that whi...","z3r0shade, I understand that you have strong ...","\nz3r0shade, it seems like you're raising some..."
345,345,Assume you are a forum moderator. Please respo...,"BeansforHomerclese, it's important to remember...","BeansforHomerclese, I understand your perspec...","\nBeansforHomerclese, it's important to rememb..."


In [78]:
generated_modertion_df = generated_modertion_df[generated_modertion_df["base_model_moderation"] != "prompt too long..."]
generated_modertion_df = generated_modertion_df.reset_index(drop=True)

In [79]:
def gpt_rate_prompt(converstion, autor, moderation_1, moderation_2, rating_scheme=1):
    if rating_scheme == 1:
        return gpt_rate_prompt_1(converstion, autor, moderation_1, moderation_2)
    elif rating_scheme == 2:
        return gpt_rate_prompt_2(converstion, autor, moderation_1, moderation_2)


def gpt_rate_prompt_1(converstion, autor, moderation_1, moderation_2):
    return f"""This is a conversation from the CMV forum:

{converstion}

{autor} got moderation messages:

The first moderation message:
---Message starts here---
{moderation_1}
---Message ends here---

The second moderation message:
---Message starts here---
{moderation_2}
---Message ends here---

Please rate the messages from 1 to 5, according to this:
1 indicates the message seems highly machine-generated and inappropriate.
3 indicates a neutral perception.
5 indicates the message seems highly human-like and appropriate.

Your respone should be in this format and only in this format:

Message 1 rating: <number>
Reason: <your reason for the rate>

Message 2 rating: <number>
Reason: <your reason for the rate>
"""


def gpt_rate_prompt_2(converstion, autor, moderation_1, moderation_2):
    return f"""This is a conversation from the CMV forum:

{converstion}

{autor} got moderation messages:

The first moderation message:
---Message starts here---
{moderation_1}
---Message ends here---

The second moderation message:
---Message starts here---
{moderation_2}
---Message ends here---


Please rate the messages from 0 to 4, according to this:
1 point for human-like and appropriate.
1 point for addressing the user that replaying to.
1 point for addressing the issue with the last comment of the user you are replying to.
1 point for a concise/short yet informative moderation message.

Your response should be in this format and only in this format:

Message 1 rating: <number>
Reason: <your reason for the rate>

Message 2 rating: <number>
Reason: <your reason for the rate>
"""

rating scheme 1

In [ ]:
import re

base_model_moderation_rates = []
fine_tuned_model_moderation_rates = []
chat_gpt_response = []

for i in range(len(generated_modertion_df)):
    # extract the converstion
    pattern = r'^Assume you are a forum moderator. Please respond to .*, as his response might require a moderation note.\s+'
    conversion = re.sub(pattern, '', generated_modertion_df["prompt"][i])

    # Find the match
    match_pos = conversion.find("Do not try to counter ")
    conversion = conversion[:match_pos]
    conversion = conversion.strip("\n")

    # get the user that need moderation node
    author = conversion.split("\n")[-1].split(":")[0]
    moderation_1 = generated_modertion_df["base_model_moderation"][i]
    moderation_2 = generated_modertion_df["fine_tuned_model_moderation"][i]

    try:
        gpt_rates = get_chatgpt_response(gpt_rate_prompt(conversion, author, moderation_1, moderation_2, rating_scheme=1))
        chat_gpt_response.append(gpt_rates)
        rate_1 = float(gpt_rates[len("Message 1 rating: "): gpt_rates.find("\n")])
        rate_2_str = gpt_rates[gpt_rates.find("Message 2 rating: ") + len("Message 2 rating: "):]
        rate_2 = float(rate_2_str[:rate_2_str.find("\n")])

        base_model_moderation_rates.append(rate_1)
        fine_tuned_model_moderation_rates.append(rate_2)
    except:
        print(i, "is two long for chatGPT")
        chat_gpt_response.append("prompt too long...")
        base_model_moderation_rates.append(np.nan)
        fine_tuned_model_moderation_rates.append(np.nan)

In [ ]:
chat_gpt_scores_df = pd.DataFrame(data={"chat_gpt_response": chat_gpt_response, "base_model_moderation_rates": base_model_moderation_rates, "fine_tuned_model_moderation_rates": fine_tuned_model_moderation_rates})

In [133]:
chat_gpt_scores_df.to_csv("test_fine_tune_chatGPT_rating_1.csv", index=False)

rating scheme 2

In [ ]:
import re

base_model_moderation_rates = []
fine_tuned_model_moderation_rates = []
chat_gpt_response = []

for i in range(len(generated_modertion_df)):
    # extract the converstion
    pattern = r'^Assume you are a forum moderator. Please respond to .*, as his response might require a moderation note.\s+'
    conversion = re.sub(pattern, '', generated_modertion_df["prompt"][i])

    # Find the match
    match_pos = conversion.find("Do not try to counter ")
    conversion = conversion[:match_pos]
    conversion = conversion.strip("\n")

    # get the user that need moderation node
    author = conversion.split("\n")[-1].split(":")[0]
    moderation_1 = generated_modertion_df["base_model_moderation"][i]
    moderation_2 = generated_modertion_df["fine_tuned_model_moderation"][i]
    if i % 50 == 0:
        print("check prompt:")
        print(gpt_rate_prompt(conversion, author, moderation_1, moderation_2))
        print("*" * 50)
    try:
        gpt_rates = get_chatgpt_response(gpt_rate_prompt(conversion, author, moderation_1, moderation_2))
        chat_gpt_response.append(gpt_rates)
        rate_1 = float(gpt_rates[len("Message 1 rating: "): gpt_rates.find("\n")])
        rate_2_str = gpt_rates[gpt_rates.find("Message 2 rating: ") + len("Message 2 rating: "):]
        rate_2 = float(rate_2_str[:rate_2_str.find("\n")])

        base_model_moderation_rates.append(rate_1)
        fine_tuned_model_moderation_rates.append(rate_2)
    except:
        print(i, "is two long for chatGPT")
        chat_gpt_response.append("prompt too long...")
        base_model_moderation_rates.append(np.nan)
        fine_tuned_model_moderation_rates.append(np.nan)

In [ ]:
chat_gpt_scores_df = pd.DataFrame(data={"chat_gpt_response": chat_gpt_response, "base_model_moderation_rates": base_model_moderation_rates, "fine_tuned_model_moderation_rates": fine_tuned_model_moderation_rates})

In [ ]:
chat_gpt_scores_df.to_csv("test_fine_tune_chatGPT_rating_2.csv")